In [1]:
import pandas as pd
df_savan=pd.read_csv("savant_data_2021_2023.csv")

In [2]:
from IPython.display import display
display(df_savan)  # Properly formatted HTML table in Jupyter

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,events,description,zone,...,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,role_key,sp_indicator,rp_indicator,pitch_number_appearance,pitcher_at_bat_number,times_faced
0,FF,2021-04-05,93.2,0.84,5.74,c7c83eaa9fe8da2f81c5fce172059af61448b3e7,4f902241478a103f7a818f5be9a7b7ddf43cabb9,strikeout,swinging_strike,13.0,...,Standard,166.0,-0.014,-0.134,SP,1,0,97,22,3
1,SL,2021-04-05,82.9,1.03,5.69,c7c83eaa9fe8da2f81c5fce172059af61448b3e7,4f902241478a103f7a818f5be9a7b7ddf43cabb9,NaN,foul,5.0,...,Standard,309.0,0.000,0.000,SP,1,0,96,22,3
2,FF,2021-04-05,94.6,-1.85,5.77,514eeb6e6c17085fc1adabf03f1adca32318411f,afb9b85defc6fe5c3f48681480eff4d96ab723c7,caught_stealing_3b,ball,14.0,...,Standard,220.0,0.014,-0.202,SP,1,0,95,19,3
3,FF,2021-04-05,97.1,1.86,6.57,875eeca87c6f80182a88c2a7b92c048b9e10b5e5,0f061b9439845159c394a71e55d635b10ca25656,field_out,hit_into_play,13.0,...,Standard,147.0,-0.007,-0.264,SP,1,0,95,22,3
4,FF,2021-04-05,93.6,0.60,5.85,c7c83eaa9fe8da2f81c5fce172059af61448b3e7,4f902241478a103f7a818f5be9a7b7ddf43cabb9,NaN,ball,11.0,...,Standard,164.0,0.000,0.043,SP,1,0,95,22,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136332,SI,2023-05-26,90.2,-1.70,3.83,ceeeda3bde736c0c656b03be05b4583f1daaf664,da2331420ceeaeed96ab885543ca5a4c6cb97958,NaN,called_strike,5.0,...,Standard,245.0,0.000,-0.038,RP,0,1,1,1,1
2136333,SL,2023-05-26,88.3,3.13,5.19,7788ff0375c64348d36ae12869506ad498eefef4,1039ef0b36a4d7d4c2e9b7b6c2084b3cf99c11dc,NaN,called_strike,8.0,...,Standard,238.0,0.000,-0.038,RP,0,1,1,1,1
2136334,FF,2023-05-26,89.7,1.10,5.73,0f46bb1ed363d593af42cfddbe027922aff10780,c6020d12585d4a0d7b1babd840c54f37a9159aea,NaN,ball,12.0,...,Standard,147.0,0.000,0.036,RP,0,1,1,1,1
2136335,FF,2023-05-26,92.6,1.21,6.25,75174627fd934205ab7d1905d926b580814f9759,9a1d0691f34a0c9c1899f3c29af36638cc7a8af6,NaN,called_strike,13.0,...,Standard,184.0,0.000,-0.063,RP,0,1,1,1,1


In [3]:
df_savan = df_savan.drop(columns=['pitcher_1', 'fielder_2_1', 'fielder_3', 'fielder_4', 'fielder_5', 
                      'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9'])

In [4]:
display(df_savan)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,events,description,zone,...,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,role_key,sp_indicator,rp_indicator,pitch_number_appearance,pitcher_at_bat_number,times_faced
0,FF,2021-04-05,93.2,0.84,5.74,c7c83eaa9fe8da2f81c5fce172059af61448b3e7,4f902241478a103f7a818f5be9a7b7ddf43cabb9,strikeout,swinging_strike,13.0,...,Standard,166.0,-0.014,-0.134,SP,1,0,97,22,3
1,SL,2021-04-05,82.9,1.03,5.69,c7c83eaa9fe8da2f81c5fce172059af61448b3e7,4f902241478a103f7a818f5be9a7b7ddf43cabb9,NaN,foul,5.0,...,Standard,309.0,0.000,0.000,SP,1,0,96,22,3
2,FF,2021-04-05,94.6,-1.85,5.77,514eeb6e6c17085fc1adabf03f1adca32318411f,afb9b85defc6fe5c3f48681480eff4d96ab723c7,caught_stealing_3b,ball,14.0,...,Standard,220.0,0.014,-0.202,SP,1,0,95,19,3
3,FF,2021-04-05,97.1,1.86,6.57,875eeca87c6f80182a88c2a7b92c048b9e10b5e5,0f061b9439845159c394a71e55d635b10ca25656,field_out,hit_into_play,13.0,...,Standard,147.0,-0.007,-0.264,SP,1,0,95,22,3
4,FF,2021-04-05,93.6,0.60,5.85,c7c83eaa9fe8da2f81c5fce172059af61448b3e7,4f902241478a103f7a818f5be9a7b7ddf43cabb9,NaN,ball,11.0,...,Standard,164.0,0.000,0.043,SP,1,0,95,22,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136332,SI,2023-05-26,90.2,-1.70,3.83,ceeeda3bde736c0c656b03be05b4583f1daaf664,da2331420ceeaeed96ab885543ca5a4c6cb97958,NaN,called_strike,5.0,...,Standard,245.0,0.000,-0.038,RP,0,1,1,1,1
2136333,SL,2023-05-26,88.3,3.13,5.19,7788ff0375c64348d36ae12869506ad498eefef4,1039ef0b36a4d7d4c2e9b7b6c2084b3cf99c11dc,NaN,called_strike,8.0,...,Standard,238.0,0.000,-0.038,RP,0,1,1,1,1
2136334,FF,2023-05-26,89.7,1.10,5.73,0f46bb1ed363d593af42cfddbe027922aff10780,c6020d12585d4a0d7b1babd840c54f37a9159aea,NaN,ball,12.0,...,Standard,147.0,0.000,0.036,RP,0,1,1,1,1
2136335,FF,2023-05-26,92.6,1.21,6.25,75174627fd934205ab7d1905d926b580814f9759,9a1d0691f34a0c9c1899f3c29af36638cc7a8af6,NaN,called_strike,13.0,...,Standard,184.0,0.000,-0.063,RP,0,1,1,1,1


In [5]:
print(df_savan.columns)

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'batter', 'pitcher', 'events', 'description', 'zone',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'fielder_2',
       'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'release_pos_y',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name',
       'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
       'post_home_score', 'post_bat_

In [ ]:
print(df_savan['events'].value_counts().to_string())

In [11]:
# Check unique values in 'events' and 'description'
print("Unique values in 'events':", df_savan['events'].unique())
# Count occurrences of each event type
print(df_savan['events'].value_counts().to_string())

Unique values in 'events': ['strikeout' nan 'caught_stealing_3b' 'field_out' 'walk' 'force_out'
 'sac_fly' 'single' 'hit_by_pitch' 'double' 'grounded_into_double_play'
 'sac_bunt' 'home_run' 'fielders_choice' 'field_error' 'other_out'
 'caught_stealing_2b' 'triple' 'strikeout_double_play'
 'fielders_choice_out' 'double_play' 'sac_fly_double_play'
 'catcher_interf' 'pickoff_caught_stealing_2b' 'pickoff_2b'
 'pickoff_caught_stealing_3b' 'triple_play' 'pickoff_1b'
 'sac_bunt_double_play' 'wild_pitch' 'game_advisory'
 'caught_stealing_home' 'pickoff_3b' 'stolen_base_2b' 'passed_ball'
 'pickoff_caught_stealing_home' 'pickoff_error_3b' 'stolen_base_3b']
events
field_out                       217075
strikeout                       124404
single                           76913
walk                             44749
double                           24031
home_run                         17027
force_out                        11153
grounded_into_double_play        10148
hit_by_pitch             

In [18]:
df_events_count = df_savan['events'].value_counts().reset_index()
df_events_count.columns = ['Event', 'Count']

display(df_events_count)


,Event,Count
0,field_out,217075
1,strikeout,124404
2,single,76913
3,walk,44749
4,double,24031
5,home_run,17027
6,force_out,11153
7,grounded_into_double_play,10148
8,hit_by_pitch,6269
9,sac_fly,3534


In [8]:
print("Unique values in 'description':", df_savan['description'].unique())
print(df_savan['description'].value_counts())


Unique values in 'description': ['swinging_strike' 'foul' 'ball' 'hit_into_play' 'blocked_ball'
 'called_strike' 'foul_tip' 'hit_by_pitch' 'foul_bunt'
 'swinging_strike_blocked' 'missed_bunt' 'pitchout' 'bunt_foul_tip'
 'foul_pitchout' 'unknown_strike']
description
ball                       712795
foul                       381153
hit_into_play              370227
called_strike              350436
swinging_strike            227015
blocked_ball                49293
foul_tip                    20996
swinging_strike_blocked     13509
hit_by_pitch                 6269
foul_bunt                    3682
missed_bunt                   760
pitchout                      119
bunt_foul_tip                  81
foul_pitchout                   1
unknown_strike                  1
Name: count, dtype: int64
